In [1]:
import geopandas as gpd
import pandas as pd
import folium
from branca.colormap import StepColormap

# Load cleaned + joined grocery store data
df = pd.read_csv("../data/cleaned/grocery_stores_v1_curated.csv")
print(f"✅ Loaded: {df.shape[0]} rows")

# Filter to only real grocery stores
real_grocery = df[df["IS_REAL_GROCERY"] == True]

# Count stores by community
grocery_counts = (
    real_grocery["Community"]
    .value_counts()
    .reset_index()
)
grocery_counts.columns = ["Community", "StoreCount"]
print("✅ Sample counts:\n", grocery_counts.head())

# Load Community Area GeoJSON
geo = gpd.read_file("../data/geo/community_areas.geojson")
geo = geo.rename(columns={"community": "Community"})  # Rename to match merge key
print("✅ Geo columns:", geo.columns.tolist())

# Merge counts into polygons
geo = geo.merge(grocery_counts, on="Community", how="left")
geo["StoreCount"] = geo["StoreCount"].fillna(0)

# Create map
m = folium.Map(location=[41.8781, -87.6298], zoom_start=11)

# Ensure numeric StoreCount
geo["StoreCount"] = pd.to_numeric(geo["StoreCount"], errors="coerce").fillna(0).astype(float)
print(geo["StoreCount"].describe())

# Updated color scale: brighter yellow for 29-39
choropleth_scale = StepColormap(
    colors=[
        "#b30000",  # 0-10 = Very red
        "#f03b20",  # 10-20 = Strong orange
        "#fc9272",  # 20-29 = Light red
        "#ffff33",  # 29-39 = Bright yellow
        "#31a354"   # 39-49+ = Green
    ],
    index=[0, 10, 20, 29, 39, 50],  # Explicit cutoffs
    vmin=0,
    vmax=50,
    caption="Number of Real Grocery Stores per Community Area"
)

# Add GeoJSON Layer
folium.GeoJson(
    geo,
    name="Grocery Store Density",
    style_function=lambda feature: {
        "fillColor": choropleth_scale(feature["properties"]["StoreCount"]) if feature["properties"]["StoreCount"] > 0 else "#f0f0f0",
        "color": "white",
        "weight": 1,
        "fillOpacity": 0.7,
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=["Community", "StoreCount"],
        aliases=["Community Area", "# of Real Grocery Stores"],
        localize=True
    )
).add_to(m)

# Add color scale
choropleth_scale.add_to(m)

# Save map
output_path = "../docs/grocery_choropleth_by_area.html"
m.save(output_path)
print(f"✅ Choropleth map saved to: {output_path}")


/Users/archangel/Desktop/chi-food-access-map/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


✅ Loaded: 4503 rows
✅ Sample counts:
          Community  StoreCount
0       WEST RIDGE          49
1   SOUTH LAWNDALE          44
2   BELMONT CRAGIN          41
3  NEAR NORTH SIDE          38
4     CHICAGO LAWN          38
✅ Geo columns: ['Community', 'shape_area', 'area_num_1', 'area_numbe', 'shape_len', 'geometry']
count    77.000000
mean     16.194805
std      12.183940
min       0.000000
25%       6.000000
50%      14.000000
75%      25.000000
max      49.000000
Name: StoreCount, dtype: float64
✅ Choropleth map saved to: ../docs/grocery_choropleth_by_area.html
